In [1]:
# Gravitational Lensing Research
# Program to take a qlens-generated sample chain, derive kappa vs. radius values. No plotting.
# Has ability to plot .dmo ("dark matter only") versions.
# This version is for A2537.

In [2]:
import time
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
import timeit
from IPython.core.debugger import set_trace
import datetime
import concurrent.futures as cf
import multiprocessing as mp

In [3]:
# set the radii values, in arc seconds, log spaced. NOTE THAT THESE MUST MATCH SCRIPT BELOW
rstart = .1
rstop  = 50
nr = 50
radii = np.logspace(np.log10(rstart), np.log10(rstop), nr, endpoint=False)

In [4]:
# for parallel processing
nproc = 10

In [5]:
name = 'A2537'
suffix = '.cnfw.v9'
dmo = True   # If doing "dark matter only", set this to True.
label = name + suffix
# Set your path here, to the "chains_..." directory of interest
#path = '/home/grant/Documents/qlens-beta/' + name + '/chains_' + label
path = '/home/kea/KEVIN/UCI/Research/gravlensing/' + name + '/chains_' + label
# path = '/Users/kevin/CloudStation/KEVIN/UCI/Research/gravlensing/' + name + '/chains_' + label

In [6]:
# now calculated the median and +/- 1-sigma values
def find_bands(p, samples):
    total = [np.sum(samples[:, i + p] * samples[:, 0]) for i in range(nr)]
    sigma_minus  = np.zeros(nr)
    sigma_plus = np.zeros(nr)
    median = np.zeros(nr)
    for i in range(nr):
        runsum = 0.
        sm_flag = sp_flag = m_flag = False
        # sort the table by increasing kappa for the radius in question
        samsort =  samples[samples[:, i + p].argsort()]
        for row in range(len(samsort)):
            runsum += (samsort[row, 0] * samsort[row, i + p]) / total[i]
            if (not sm_flag) and (runsum > 0.158):
                sm_flag = True
                sigma_minus[i] = samsort[row, i + p]
            if (not sp_flag) and (runsum > 0.841):
                sp_flag = True
                sigma_plus[i] = samsort[row, i + p] 
            if (not m_flag) and (runsum > 0.5):
                m_flag = True
                median[i] = samsort[row, i + p]
    return sigma_minus, sigma_plus, median

In [7]:
def ql_script(i, p, params, tstart, ns, dmo):
    # assembles and runs the QLens script, returning a vector of kappa values
    # Note that the first and pth columns are not parameters.
    # the pth column is the chi square.
    #like,  mvir, c, rc_kpc, q, theta, xc, yc, shear1, shear2, mtot1, mtot2, raw_chisq, chisq = params[0:p] 
    
    like, mvir, c, rc_kpc, q, theta, xc, yc, shear1, shear2, mtot1, mtot2, raw_chisq, chisq = params[0:p]
    
    pars0 = ' '.join([str(item) for item in [mvir, c, rc_kpc, q, theta, xc, yc]]) # Halo parameters
    
    pars1 = ' '.join([str(item) for item in [shear1, shear2]]) # Shear parameters

    pars2 = ' '.join([str(item) for item in [mtot1, 52.7, 0.75, 0.72951, 31, -0, 0.0001]]) # BCG parameters

    pars3 = ' '.join([str(item) for item in [mtot2, 44.004, 0.62624, 1, 1, -44.003, -23.72 ] ]) # GR parameters

    kappaname = 'plotdata/kappadist_temp_' + str(mp.current_process().name) + '.dat'

    script_part_a1 = '''
    # skeleton script to load bestfit results
    zlens=0.294
    zsrc=2.0
    zsrc_ref=2.0
    fit method multinest
    shear_components on
    major_axis_along_y off    
    central_image on  # REVIEW
    galsubgrid on
    gridtype cartesian
    n_livepts = 4000 
    chisqmag on 
    chisqtol 1e-6 
    chisq_time_delays off 
    analytic_bestfit_src on	
    imgplane_chisq off
    sci_notation off
    chisq_imgsep_threshold = 1e-3  
    nimg_penalty off                 
    chisq_mag_threshold = 0.1  
    analytic_bestfit_src on	
    gridtype cartesian
    grid -80 80  -80  80
    imgdata read ../A2537_SL_Data.dat   #REVIEW
    '''

    script_part_a2 = '''
    lens clear
    lens cnfw pmode=3 '''
    # here we insert the parameters for the first lens (halo)

    script_part_b1 =''' shear='''
    # here we insert the paramters for the shear
    
    script_part_b2 ='''
    lens pjaffe pmode=2 '''
    # here we insert the parameters for the second lens (BCG)

    script_part_b3 ='''
    lens pjaffe pmode=2 '''
    # here we insert the params for anchor galaxy

    script_part_c = '''
lens pjaffe pmode=2 1.102e+12/anchor=3,0 53.865/anchor=3,1 0.76659/anchor=3,2 1.000 1.000 -37.92240 -41.91840 # ID 1
lens pjaffe pmode=2 5.895e+11/anchor=3,0 39.398/anchor=3,1 0.56069/anchor=3,2 1.000 1.000 -1.47240 12.34800 # ID 4
lens pjaffe pmode=2 5.211e+11/anchor=3,0 37.040/anchor=3,1 0.52713/anchor=3,2 1.000 1.000 -24.44040 -4.79880 # ID 5
lens pjaffe pmode=2 5.052e+11/anchor=3,0 36.471/anchor=3,1 0.51904/anchor=3,2 1.000 1.000 -24.91200 -37.69920 # ID 6
lens pjaffe pmode=2 4.976e+11/anchor=3,0 36.199/anchor=3,1 0.51516/anchor=3,2 1.000 1.000 38.35800 12.85920 # ID 7
lens pjaffe pmode=2 4.389e+11/anchor=3,0 33.993/anchor=3,1 0.48377/anchor=3,2 1.000 1.000 -42.24240 17.24760 # ID 8
lens pjaffe pmode=2 3.4e+11/anchor=3,0 29.921/anchor=3,1 0.42582/anchor=3,2 1.000 1.000 -16.77240 9.54720 # ID 9
lens pjaffe pmode=2 3.005e+11/anchor=3,0 28.129/anchor=3,1 0.40031/anchor=3,2 1.000 1.000 12.61080 13.73760 # ID 10
lens pjaffe pmode=2 2.912e+11/anchor=3,0 27.692/anchor=3,1 0.39410/anchor=3,2 1.000 1.000 35.75520 14.06520 # ID 11
lens pjaffe pmode=2 2.823e+11/anchor=3,0 27.265/anchor=3,1 0.38803/anchor=3,2 1.000 1.000 -10.79640 10.86480 # ID 12
lens pjaffe pmode=2 2.606e+11/anchor=3,0 26.193/anchor=3,1 0.37277/anchor=3,2 1.000 1.000 14.79960 49.98600 # ID 13
lens pjaffe pmode=2 2.542e+11/anchor=3,0 25.871/anchor=3,1 0.36818/anchor=3,2 1.000 1.000 -30.49920 13.90320 # ID 15
lens pjaffe pmode=2 2.41e+11/anchor=3,0 25.192/anchor=3,1 0.35853/anchor=3,2 1.000 1.000 -10.42920 20.94480 # ID 16
lens pjaffe pmode=2 1.959e+11/anchor=3,0 22.713/anchor=3,1 0.32324/anchor=3,2 1.000 1.000 -52.85880 -16.81920 # ID 18
lens pjaffe pmode=2 1.83e+11/anchor=3,0 21.952/anchor=3,1 0.31241/anchor=3,2 1.000 1.000 3.60720 3.94200 # ID 22
lens pjaffe pmode=2 1.811e+11/anchor=3,0 21.839/anchor=3,1 0.31080/anchor=3,2 1.000 1.000 35.21880 48.43080 # ID 23
lens pjaffe pmode=2 1.781e+11/anchor=3,0 21.654/anchor=3,1 0.30816/anchor=3,2 1.000 1.000 17.43840 41.40720 # ID 24
lens pjaffe pmode=2 1.706e+11/anchor=3,0 21.193/anchor=3,1 0.30161/anchor=3,2 1.000 1.000 37.39680 24.12360 # ID 25
lens pjaffe pmode=2 1.689e+11/anchor=3,0 21.086/anchor=3,1 0.30008/anchor=3,2 1.000 1.000 -6.04440 23.00760 # ID 26
lens pjaffe pmode=2 1.672e+11/anchor=3,0 20.981/anchor=3,1 0.29859/anchor=3,2 1.000 1.000 -4.60440 -10.85040 # ID 27
lens pjaffe pmode=2 1.575e+11/anchor=3,0 20.365/anchor=3,1 0.28983/anchor=3,2 1.000 1.000 -20.44080 16.54200 # ID 28
lens pjaffe pmode=2 1.561e+11/anchor=3,0 20.275/anchor=3,1 0.28854/anchor=3,2 1.000 1.000 37.14120 37.41480 # ID 29
lens pjaffe pmode=2 1.555e+11/anchor=3,0 20.237/anchor=3,1 0.28801/anchor=3,2 1.000 1.000 -47.58480 -15.69600 # ID 30
lens pjaffe pmode=2 1.51e+11/anchor=3,0 19.942/anchor=3,1 0.28381/anchor=3,2 1.000 1.000 48.15360 -2.55240 # ID 31
lens pjaffe pmode=2 1.422e+11/anchor=3,0 19.353/anchor=3,1 0.27542/anchor=3,2 1.000 1.000 13.12920 14.37480 # ID 32
lens pjaffe pmode=2 1.369e+11/anchor=3,0 18.985/anchor=3,1 0.27019/anchor=3,2 1.000 1.000 44.38800 -18.60120 # ID 33
lens pjaffe pmode=2 1.303e+11/anchor=3,0 18.520/anchor=3,1 0.26357/anchor=3,2 1.000 1.000 30.48480 24.01200 # ID 35
lens pjaffe pmode=2 1.275e+11/anchor=3,0 18.322/anchor=3,1 0.26075/anchor=3,2 1.000 1.000 -3.42360 -17.33760 # ID 36
lens pjaffe pmode=2 1.055e+11/anchor=3,0 16.669/anchor=3,1 0.23723/anchor=3,2 1.000 1.000 -13.16520 -37.93320 # ID 37
lens pjaffe pmode=2 1.024e+11/anchor=3,0 16.418/anchor=3,1 0.23366/anchor=3,2 1.000 1.000 24.56640 42.08760 # ID 39
lens pjaffe pmode=2 1.02e+11/anchor=3,0 16.387/anchor=3,1 0.23322/anchor=3,2 1.000 1.000 -38.19600 16.03440 # ID 40
 
    plotkappa ''' + str(rstart) + ' ' + str(rstop) + ' ' + str(nr) + ' '

    
    script = script_part_a1 + script_part_a2 + pars0 + script_part_b1 + pars1\
        + script_part_b2 + pars2 + script_part_b3 + pars3  \
        + script_part_c + kappaname + (' ' if not dmo else ' lens=0 ') + '\n'
    sc_name = 'plotdata/kappascript_' + str(mp.current_process().name) + '.in'

    with open(sc_name, 'w') as outfile:
        outfile.write(script)

    # Now, run the script
    os.system('qlens ' + sc_name + ' -q')
    # read in kappa file back in, and store results
    kdat = np.loadtxt(kappaname)
    kappa = kdat[:, 1]
    if i==0:
        rad = kdat[:, 0]
        assert np.all(np.abs((rad - radii) / radii) < .001)
    if i%50 == 1:
        tavg = (timeit.default_timer() - tstart) / (i+1.)
        proj_fin =  tavg * (ns -i -1)
        sys.stdout.write("\r{0:5.2%} complete. Avg time per iteration: {1:8.6f} sec. Projected finish in {2:4.2f} min. "\
                   .format((float(i) / ns), tavg,  proj_fin/60))
    sys.stdout.flush() # required in multiprocessing or else it buffers this output
    return kappa

In [8]:
# Function to load cNFW chain, create the script, run it, find the bands, and return the new samples and bands.
def do_script(label, path, dmo= False): # the path should be the /chains_... directory
    os.chdir(path)
    print(os.getcwd())
    # Import the chain
    samples = np.loadtxt(path + '/' + label, comments="#", delimiter=None, unpack=False)
    with open(path + '/' + label + '.paramnames') as afile:
        paramnames = afile.readlines()
    paramnames = [item[:-1] for item in paramnames]  # Strip off the \n character.
    samples = np.array(samples)
    ns, p = samples.shape
    print(samples.shape)
    print(paramnames)

    # extend the samples to have nr additional columns
    samples2 = np.append(samples, np.zeros((ns, nr)), axis=1)
    samples2.shape
    # delete all samples with zero weight
    mask = [False if sam[0]==0. else True for sam in samples2]
    samples2 = samples2[mask]
    print(samples2.shape)
    ns = samples2.shape[0]
    sort = samples2[samples2[:,0].argsort()]  # This sorts the samples in ascending order, based on weight (first col.)
 
    # make sure there is a directory called plotdata. If not, create it.
    if not os.path.exists(os.getcwd()+'/plotdata'):
        os.makedirs(os.getcwd()+'/plotdata')
    
    results = []
    tstart = timeit.default_timer()
    with cf.ProcessPoolExecutor(nproc) as pp:
        for i, params in enumerate(samples2[:, 0:p]):
            results.append((i, pp.submit(ql_script, i, p, params, tstart, ns, dmo=dmo)))
    # code waits here until all processes are finished
    print("\nKappa calculation completed in ", (timeit.default_timer() - tstart)/60, " minutes.                          ")
    
    old_i = -1
    for (i,k) in results:
        samples2[i, p:] = k.result()
        if old_i +1 != i:
            print('WARNING: Out of order at i=', i)
        old_i = i
    print('Results saved into samples array.')  
    
    sm, sp, med = find_bands(p, samples2)
    return samples2, sm, sp, med
    

In [9]:

samp, sigma_minus, sigma_plus, median = do_script(label, path, dmo)

/home/kea/KEVIN/UCI/Research/gravlensing/A2537/chains_A2537.cnfw.v9
(118979, 14)
['mvir', 'c', 'rc_kpc', 'q', 'theta', 'xc', 'yc', 'shear1', 'shear2', 'mtot1', 'mtot2', 'raw_chisq']
(118979, 64)
99.98% complete. Avg time per iteration: 0.017789 sec. Projected finish in 0.01 min.  
Kappa calculation completed in  35.27485945864998  minutes.                          
Results saved into samples array.


In [14]:
np.savez(label + ('.dmo' if dmo else '') + '.bands', radii, sigma_minus, sigma_plus, median)

In [ ]:
# Use Notebook "Kappa band plotter" to plot the results.